In [ ]:
!pip install -qr "../requirements.txt"

In [ ]:
from datasets import load_dataset

import torch
import torch.nn as nn
import torch.nn.functional as F

import word_embedding_classifier as model
import word_embedding_processor as processor
import word_embedding_evaluation as evaluation

import glob
import pandas as pd

In [ ]:
arc_dataset = load_dataset('ai2_arc', 'ARC-Easy')

device = 'cuda' if torch.cuda.is_available() else 'cpu'

criterion = nn.CrossEntropyLoss() 

In [ ]:
def test_model(hidden_dim, num_layers, activation_function, network_file, inputs, labels):
    net = model.Word_embedding_classifier(100, hidden_dim, 4, num_layers, activation_function).to(device)
    net.load_state_dict(network_file)
    net.eval()

    labels = torch.asarray(labels).to(device).type(torch.float32)
    inputs = inputs.to(device).type(torch.float32)

    with torch.no_grad():
        outputs = net(inputs[:,0], inputs[:,1], inputs[:,2], inputs[:,3], inputs[:,4])
        loss = criterion(outputs, labels)
        accuracy_score = evaluation.calculate_accuracy_from_predicitions(outputs, labels)

    return loss, accuracy_score

In [ ]:
X_test_with_stopwords, y_test_with_stopwords = processor.process_dataset(arc_dataset['test'],remove_stop_words=False)
X_test_without_stopwords, y_test_without_stopwords = processor.process_dataset(arc_dataset['test'],remove_stop_words=True)

In [ ]:
trial_number = 66
path = f'models/3-Run/'
filename = 'fast_text_classifier_trial_'

df = pd.read_csv(f'{path}{filename}{trial_number}.csv')

row = df.iloc[0]

trial_number = row['Trial number']
hidden_dim = row['hidden_dim']
num_layers = row['num_layers']
dataset = row['dataset']
activation_function = row['activation_function']
word_embedding = row['word_embedding']

if dataset == 'with_stopwords':
    X_test, y_test = X_test_with_stopwords, y_test_with_stopwords
elif dataset == 'without_stopwords':
    X_test_trial, y_test_trial = X_test_without_stopwords, y_test_without_stopwords

network = torch.load(f'{path}{filename}{trial_number}.pth', map_location=device)

loss, accuracy_score = test_model(hidden_dim, num_layers, getattr(F, activation_function), network, X_test_trial, y_test_trial)
    
print(row)
print(f'Train accuracy of best trial: {accuracy_score}')

print(f'Train loss of best trial: {loss}')